In [115]:
import pandas as pd

In [116]:
table = pd.read_excel('/user_data.xlsx')
table.head()

,userID,eventName,time,product
0,1,buy,2023-03-01 18:56:09,p1
1,1,buy,2023-03-06 10:58:59,p2
2,1,download,2023-03-02 10:58:59,p1
3,1,download,2023-03-10 11:56:09,p2
4,1,register,2023-03-01 10:58:59,NaN


Находим время регистрации пользователя

In [117]:
#вариант 1
#register_time = table.query('time == register_time')
#register_time = register_time.reindex(['userID', 'register_time'], axis=1)
#table = table.merge(register_time, how='left', on = 'userID')
#table.head()

In [118]:
#вариант 2
table['register_time'] = table.groupby('userID')['time'].transform(
    lambda x: x[table.loc[x.index, 'eventName'] == 'register'].min())
table.head()

,userID,eventName,time,product,register_time
0,1,buy,2023-03-01 18:56:09,p1,2023-03-01 10:58:59
1,1,buy,2023-03-06 10:58:59,p2,2023-03-01 10:58:59
2,1,download,2023-03-02 10:58:59,p1,2023-03-01 10:58:59
3,1,download,2023-03-10 11:56:09,p2,2023-03-01 10:58:59
4,1,register,2023-03-01 10:58:59,NaN,2023-03-01 10:58:59


Определяем номер недели для каждого действия и номер недели даты регистрации для формирования когорт

In [119]:
table['week'] = table['time'].dt.to_period('W')

In [120]:
table['register_week'] = table['register_time'].dt.to_period('W')

In [121]:
table['n_week'] = table['week'].rank(method = 'dense', ascending = 1).astype(int)

In [122]:
table['n_register_week'] = table['register_week'].rank(method = 'dense', ascending = 1).astype(int)

In [123]:
table['week_from_reg'] = (table['n_week'] - table['n_register_week']).astype(int)
table.head()

,userID,eventName,time,product,register_time,week,register_week,n_week,n_register_week,week_from_reg
0,1,buy,2023-03-01 18:56:09,p1,2023-03-01 10:58:59,2023-02-27/2023-03-05,2023-02-27/2023-03-05,1,1,0
1,1,buy,2023-03-06 10:58:59,p2,2023-03-01 10:58:59,2023-03-06/2023-03-12,2023-02-27/2023-03-05,2,1,1
2,1,download,2023-03-02 10:58:59,p1,2023-03-01 10:58:59,2023-02-27/2023-03-05,2023-02-27/2023-03-05,1,1,0
3,1,download,2023-03-10 11:56:09,p2,2023-03-01 10:58:59,2023-03-06/2023-03-12,2023-02-27/2023-03-05,2,1,1
4,1,register,2023-03-01 10:58:59,NaN,2023-03-01 10:58:59,2023-02-27/2023-03-05,2023-02-27/2023-03-05,1,1,0


Теперь можем посчитать CR в загрузки

In [124]:
cohort_data = table[['userID', 'eventName', 'n_register_week', 'week_from_reg']] # формируем датасет с нужными колонками
cohort_data = cohort_data[cohort_data['eventName'].isin(['download', 'register'])] # так как нас интереует CR в загрузки то мы оставляем только соответвующие действия юзеров
cohort_data.head(8)

,userID,eventName,n_register_week,week_from_reg
2,1,download,1,0
3,1,download,1,1
4,1,register,1,0
6,2,download,1,1
7,2,register,1,0
9,3,download,1,0
10,3,download,1,1
11,3,register,1,0


По условию помним, что нужно посчитать CR по каждой когорте для пользователей которые скачали приложение в первую неделю регистрации

In [125]:
CR_data = cohort_data.groupby('n_register_week', as_index=False) \
                     .agg(reg_users=('userID', lambda x: x[cohort_data['eventName'] == 'register'].nunique()),
                          download_users=('userID', lambda x: x[cohort_data['eventName'] == 'download'][cohort_data['week_from_reg'] == 0].nunique()))

CR_data['CR'] = (CR_data['download_users'] / CR_data['reg_users']).round(2)
CR_data # [['n_register_week', 'CR']]

,n_register_week,reg_users,download_users,CR
0,1,7,3,0.43
1,2,10,4,0.40
2,3,4,0,0.00
3,4,3,0,0.00
4,5,3,0,0.00


In [126]:
CR_data[['n_register_week', 'CR']]

,n_register_week,CR
0,1,0.43
1,2,0.40
2,3,0.00
3,4,0.00
4,5,0.00
